In [2]:
import os
from dataclasses import dataclass, field
from typing import Dict, Optional
import pickle

import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from transformers import AutoTokenizer, HfArgumentParser, TrainingArguments, AutoModelForCausalLM
from transformers import pipeline
import pickle





# print(prompt)


import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "meta-llama/Llama-2-13b-hf"
adapters_name = "/app/azureuser/trl/results/checkpoint-900"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    # load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 3
stop_token_ids = [0]

Starting to load the model meta-llama/Llama-2-13b-hf into memory


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data_dir = "demo_pairs/demo_99.pkl"
with open(data_dir, "rb") as f:
    data_dict = pickle.load(f)
    total_samples = len(data_dict["prompt"])
    
prompt = data_dict["prompt"][200]

inputs = tokenizer(
    prompt,
    return_tensors="pt",
)

device = torch.device("cuda:0")
m = m.to(device)

print(inputs)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = m.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=3, eos_token_id=3, bos_token_id=1,
        return_dict_in_generate=True, output_scores=True)
    
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
    # generated_sequences = outputs.sequences[:, inputs["input_ids"].shape[-1]:]
    # probabilities = torch.stack(outputs.scores, dim=1).softmax(-1)
    # texts_probabilities = torch.gather(probabilities, 2, generated_sequences[:, :, None]).squeeze(-1)
    # _scores = texts_probabilities.prod(-1)


{'input_ids': tensor([[    1, 29871,    13,  ..., 29896, 29900, 29900]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


AttributeError: 'SampleDecoderOnlyOutput' object has no attribute 'detach'

In [ ]:
_scores

tensor([1.], device='cuda:0')